# Estimate the depreciation, MSRP, and Excise Taxes

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textlog import textSetup
text = textSetup(prefix='Depreciation: ')

In [2]:
vehicles = pd.read_csv('../data/vehicles_insurance.csv')

In [50]:
depreciation = pd.read_excel('../Blue Book depreciation of cars.xlsx', sheet_name='Numbers to Use').set_index('Age')

In [51]:
depreciation

,CAR_US,CAR_FOR,Car,Pickup,Convertible,SUV,Van,Other,Wagon
Age,,,,,,,,,
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.685487,0.877813,0.781650,0.883494,0.885447,0.978918,0.859405,0.841733,0.781650
2,0.631257,0.726675,0.678966,0.678185,0.743738,0.929139,0.768279,0.729401,0.678966
3,0.592277,0.650052,0.621165,0.656546,0.649714,0.883055,0.628825,0.662850,0.621165
4,0.576336,0.595106,0.585721,0.575231,0.607960,0.632934,0.571974,0.591373,0.585721
5,0.393851,0.542197,0.468024,0.516978,0.557320,0.558901,0.476033,0.497666,0.468024
6,0.319948,0.492348,0.406148,0.352926,0.507788,0.468674,0.418396,0.421547,0.406148
7,0.188326,0.444473,0.316400,0.327768,0.449110,0.380051,0.348063,0.346324,0.316400
8,0.163138,0.373982,0.268560,0.290143,0.410202,0.321019,0.312825,0.301054,0.268560


In [8]:
vehicles['depreciationType'] = vehicles.kind.replace({
    'Other Truck': 'Pickup',
    'Something Else': np.nan
}).astype('category')

vehicles = vehicles[~pd.isnull(vehicles.depreciationType)] # drop "Something Else"

In [52]:
depreciation = pd.DataFrame(depreciation.stack()).reset_index()\
    .rename(columns={'Age': 'age', 'level_1': 'depreciationType', 0: 'depreciationValueProportion'})


In [53]:
vehiclesWithDepreciation = vehicles.copy()
vehiclesWithDepreciation['depreciationAge'] = 2018 - vehiclesWithDepreciation.VEHYEAR
vehiclesWithDepreciation['prevDepreciationAge'] = vehiclesWithDepreciation.depreciationAge - 1
vehiclesWithDepreciation = vehiclesWithDepreciation.merge(
    depreciation,
    left_on=['depreciationAge', 'depreciationType'],
    right_on=['age', 'depreciationType'],
    how='left',
    validate='m:1'
)

vehiclesWithDepreciation = vehiclesWithDepreciation.merge(
    depreciation.rename(columns={'depreciationValueProportion': 'prevDepreciationValueProportion'}),
    left_on=['prevDepreciationAge', 'depreciationType'],
    right_on=['age', 'depreciationType'],
    how='left',
    validate='m:1'
)
    
vehiclesWithDepreciation['prevYearValue'] =\
    vehiclesWithDepreciation.prevDepreciationValueProportion / vehiclesWithDepreciation.depreciationValueProportion *\
    vehiclesWithDepreciation.value_imputed
vehiclesWithDepreciation['msrp'] = (1 / vehiclesWithDepreciation.depreciationValueProportion *\
    vehiclesWithDepreciation.value_imputed).fillna(0)
vehiclesWithDepreciation['annualDepreciation'] =\
    (vehiclesWithDepreciation.prevYearValue - vehiclesWithDepreciation.value_imputed).fillna(0)


In [62]:
vehiclesWithDepreciation.groupby('depreciationAge').msrp.apply(np.median)

depreciationAge
1       32190.116504
2       31463.147782
3       30350.999401
4       29998.910348
5       32634.048551
6       33783.938185
7       37965.484558
8       39067.687642
9       39762.628118
10      47530.590940
11      51186.466920
12      56347.031090
13      60517.657994
14      64754.155861
15          0.000000
16          0.000000
17          0.000000
18          0.000000
19          0.000000
20          0.000000
21          0.000000
22          0.000000
23          0.000000
24          0.000000
25          0.000000
26          0.000000
27          0.000000
28          0.000000
29          0.000000
30          0.000000
31          0.000000
32          0.000000
33          0.000000
41          0.000000
2025        0.000000
2026        0.000000
Name: msrp, dtype: float64

## Compute Indiana excise taxes

In [ ]:
indianaTaxTable = pd.read_excel('../IN reg fees.xlsx', skiprows=1)
indianaTaxTable

In [ ]:
def computeIndianaExciseTaxes (vehicle):
    

vehicles['regfee'] = 0
indiana = vehicles.HHSTATE == 'IN'
vehicles.loc[indiana, 'regfee'] = vehicles[indiana].apply()